# Setup

In [4]:
import warnings, os
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib import colors
import pandas as pd
import numpy as np
import palettable

from astropy.table import Table
import astropy.units as u
import astropy.coordinates as coords
from corner import corner
from tap import GaiaArchive

import neighbors
from neighbors.visualization import plot_cov_ellipse
from neighbors.tools import make_cov
import hiso

plt.rc("figure", dpi=120)
plt.style.use('paperfigure.mplstyle')

In [9]:
gaia = GaiaArchive()
# gaia.login('soh')

## Functions on df

In [2]:
def make_icrs(df, ignore_velocity=False, radial_velocity_fill=None):
    """Make astropy coordinates from gaia_source DataFrame
    
    ignore_velocity : bool, False
        Use ra, dec, parallax only.
    radial_velocity_fill : float, None
        If not None, substitute this value for RV nulls.
    
    Returns astropy coordinates
    """
    if ignore_velocity:
        return coords.ICRS(
            ra=df.ra.values*u.deg,        
            dec=df.dec.values*u.deg,
            distance=1000./df.parallax.values*u.pc)
    if radial_velocity_fill is None:
        rv = df.radial_velocity.values
    else:
        rv = df.radial_velocity.fillna(value=radial_velocity_fill).values
    return coords.SkyCoord(
            ra=df.ra.values*u.deg,        
            dec=df.dec.values*u.deg,
            distance=1000./df.parallax.values*u.pc,
            pm_ra_cosdec=df.pmra.values*u.mas/u.year,
            pm_dec=df.pmdec.values*u.mas/u.year,
            radial_velocity=rv*u.km/u.s)

In [14]:
def add_xv(df, frame, unit=u.pc):
    """
    Add cartesian coordinates x, y, z, vx, vy, vz for a given `frame`
    
    df : pd.DataFrame
        Gaia DR2 data
    frame : astropy coordinate frame
        Frame to calculate coordinates in
    
    Returns df with x, y, z, vx, vy, vz columns added.
    """
    df = df.copy()
    c = make_icrs(df).transform_to(frame)
    df['x'], df['y'], df['z'] = c.cartesian.xyz.to(u.pc).value
    df['vx'], df['vy'], df['vz'] = c.velocity.d_xyz.value
    return df

In [15]:
def extract_a_g_error(df):
    """
    Extract lerr and uerr of a_g from percentiles.
    Returns df with a_g_lerr, a_g_uerr columns added.
    """
    df = df.copy()
    lerr = df['a_g_val'] - df['a_g_percentile_lower']
    uerr = df['a_g_percentile_upper'] - df['a_g_val']
    df['a_g_lerr'], df['a_g_uerr'] = lerr, uerr
    return df

In [16]:
def add_distmod(df):
    df = df.copy()
    df['distmod'] = 5*np.log10(df['parallax']) -10
    
def add_gMag(df):
    df = df.copy()
    df['gMag'] = df['phot_g_mean_mag'] + 5*np.log10(df['parallax']) - 10
    return df

In [17]:
def flag_good_phot(df):
    df = df.copy()
    good_phot = ((df.phot_bp_rp_excess_factor > 1+0.015*df.bp_rp**2) & (df.phot_bp_rp_excess_factor < 1.3+0.06*df.bp_rp**2))
    df['good_phot'] = good_phot
    return df

In [ ]:
def flag_excess(df):
    df = df.copy()
    ee = np.zeros(len(df)).astype(bool)
    ee[(df.bp_rp>1.8)\
       & ((df.bp_rp-1.8)*0.65<df.astrometric_excess_noise)\
       &(df.astrometric_excess_noise_sig>3)] = True
    ee[(df.bp_rp<1.8) \
       & (df.astrometric_excess_noise>0)\
       & (df.astrometric_excess_noise_sig>3)] = True
    df['excess_excess'] = ee
    return df

In [20]:
def table_to_pandas(t):
    """Convert astropy table to pandas dataframe.
    When source_id is converted to float, significant digits are lost, which
    result in duplicated source_id which is actually unique. To prevent this,
    when the column dtype is int and there is no missing values, keep dtype int
    instead of converting to nan.
    NOTE: This conversion is because np.nan is not supported for int types.
    """
    from collections import OrderedDict
    out = OrderedDict()
    for name, column in t.columns.items():
        if column.dtype.kind in ['i', 'u']:
            if column.mask.sum() == 0:
                out[name] = column.astype(int)
            else:
                out[name] = column.astype(float).filled(np.nan)
        elif column.dtype.kind in ['f', 'c']:
            out[name] = column.filled(np.nan)
        else:
            out[name] = column.astype(object).filled(np.nan)


        if out[name].dtype.byteorder not in ('=', '|'):
            out[name] = out[name].byteswap().newbyteorder()

    return pd.DataFrame(out)

In [18]:
# Load DR1 and DR2 data for Oh17 stars
%cd ~/projects/spelunky

dr1 = pd.read_csv("oh17star_dr1_tgas.csv")
dr2 = pd.read_csv("oh17star_dr2.csv")

%cd -

/Users/semyeong/projects/zeroday
/Users/semyeong/projects/Group10/notebooks


In [19]:
# Get group of interest
g = dr1.groupby('group_id').get_group(10)
g_dr2 = dr2.groupby('group_id').get_group(10)

# Query box in astrometric parameters

## Query position

In [18]:
cutparams = dict(
    parallax_min=8.3, parallax_max=12.4,
    ra_min=g.ra.mean()-20, ra_max=g.ra.mean()+20,
    dec_min=g.dec.mean()-9, dec_max=g.dec.mean()+9
)
query_region = """
select *
from gaiadr2.gaia_source
where
    ra between {ra_min} and {ra_max}
    and dec between {dec_min} and {dec_max}
    and parallax between {parallax_min} and {parallax_max}
""".format(**cutparams)


%store -r g10_region
try:
    g10_region
    print("Loaded g10_region from cache")
except NameError:
    print("Querying Gaia archive")
    print(query_region)
    j = gaia.query(query_region)
    g10_region = table_to_pandas(j)
    %store g10_region

Loaded g10_region from cache


## Get tmass and allwise photometry

In [13]:
cutparams = dict(
    parallax_min=8.3, parallax_max=12.4,
    ra_min=g.ra.mean()-20, ra_max=g.ra.mean()+20,
    dec_min=g.dec.mean()-9, dec_max=g.dec.mean()+9
)
query_region_phot = """
select gaiadr2.gaia_source.designation, gaiadr2.gaia_source.source_id, allwise.*, tmass.*
from gaiadr2.gaia_source
-- allwise
left join gaiadr2.allwise_best_neighbour on gaiadr2.allwise_best_neighbour.source_id=gaiadr2.gaia_source.source_id
left join gaiadr1.allwise_original_valid as allwise on allwise.allwise_oid = gaiadr2.allwise_best_neighbour.allwise_oid
-- tmass
left join gaiadr2.tmass_best_neighbour on gaiadr2.tmass_best_neighbour.source_id=gaiadr2.gaia_source.source_id
left join gaiadr1.tmass_original_valid tmass on tmass.tmass_oid = gaiadr2.tmass_best_neighbour.tmass_oid
where
    gaiadr2.gaia_source.ra between {ra_min} and {ra_max}
    and gaiadr2.gaia_source.dec between {dec_min} and {dec_max}
    and gaiadr2.gaia_source.parallax between {parallax_min} and {parallax_max}
""".format(**cutparams)


%store -r g10_region_phot
try:
    g10_region_phot
    print("Loaded g10_region_phot from cache {:d} rows".format(len(g10_region_phot)))
except NameError:
    print("Querying Gaia archive")
    print(query_region_phot)
    j = gaia.query(query_region_phot)
    g10_region_phot = j.to_pandas()
    %store g10_region_phot

Loaded g10_region_phot from cache 3413 rows


## Get panstarrs

In [14]:
cutparams = dict(
    parallax_min=8.3, parallax_max=12.4,
    ra_min=g.ra.mean()-20, ra_max=g.ra.mean()+20,
    dec_min=g.dec.mean()-9, dec_max=g.dec.mean()+9
)
query_region_ps = """
select gaiadr2.gaia_source.designation, gaiadr2.gaia_source.source_id,
  ps.*
from gaiadr2.gaia_source
-- panstarrs
left join gaiadr2.panstarrs1_best_neighbour
  on gaiadr2.panstarrs1_best_neighbour.source_id=gaiadr2.gaia_source.source_id
left join gaiadr2.panstarrs1_original_valid as ps
  on ps.obj_id = gaiadr2.panstarrs1_best_neighbour.original_ext_source_id
where
    gaiadr2.gaia_source.ra between {ra_min} and {ra_max}
    and gaiadr2.gaia_source.dec between {dec_min} and {dec_max}
    and gaiadr2.gaia_source.parallax between {parallax_min} and {parallax_max}
""".format(**cutparams)


%store -r g10_region_ps
try:
    g10_region_ps
    print("Loaded g10_region_ps from cache {:d} rows".format(len(g10_region_ps)))
except NameError:
    print("Querying Gaia archive")
    print(query_region_ps)
    j = gaia.query(query_region_ps)
    g10_region_ps = j.to_pandas()
    %store g10_region_ps

Loaded g10_region_ps from cache 3413 rows


In [41]:
cutparams = dict(
    parallax_min=8.3, parallax_max=12.4,
    ra_min=g.ra.mean()-20, ra_max=g.ra.mean()+20,
    dec_min=g.dec.mean()-9, dec_max=g.dec.mean()+9
)
query_region_dr1 = """
select gaiadr2.gaia_source.source_id, gaiadr2.dr1_neighbourhood.*
from gaiadr2.gaia_source
-- dr1
left join gaiadr2.dr1_neighbourhood
  on gaiadr2.dr1_neighbourhood.dr2_source_id=gaiadr2.gaia_source.source_id
where
    gaiadr2.gaia_source.ra between {ra_min} and {ra_max}
    and gaiadr2.gaia_source.dec between {dec_min} and {dec_max}
    and gaiadr2.gaia_source.parallax between {parallax_min} and {parallax_max}
""".format(**cutparams)


%store -r g10_region_dr1
try:
    g10_region_dr1
    print("Loaded g10_region_dr1 from cache {:d} rows".format(len(g10_region_dr1)))
except NameError:
    print("Querying Gaia archive")
    print(query_region_dr1)
    j = gaia.query(query_region_dr1)
    g10_region_dr1 = table_to_pandas(j)
    %store g10_region_dr1

no stored variable g10_region_dr1
Querying Gaia archive

select gaiadr2.gaia_source.source_id, gaiadr2.dr1_neighbourhood.*
from gaiadr2.gaia_source
-- dr1
left join gaiadr2.dr1_neighbourhood
  on gaiadr2.dr1_neighbourhood.dr2_source_id=gaiadr2.gaia_source.source_id
where
    gaiadr2.gaia_source.ra between 195.32044234355484 and 235.32044234355484
    and gaiadr2.gaia_source.dec between 47.13366454840455 and 65.13366454840455
    and gaiadr2.gaia_source.parallax between 8.3 and 12.4

Stored 'g10_region_dr1' (DataFrame)


## Check RAVE DR5 matches

In [15]:
cutparams = dict(
    parallax_min=8.3, parallax_max=12.4,
    ra_min=g.ra.mean()-20, ra_max=g.ra.mean()+20,
    dec_min=g.dec.mean()-9, dec_max=g.dec.mean()+9
)
query_region_ravedr5 = """
select gaiadr2.gaia_source.designation, gaiadr2.gaia_source.source_id, gaiadr2.ravedr5_best_neighbour.*
from gaiadr2.gaia_source
-- ravedr5
left join gaiadr2.ravedr5_best_neighbour on gaiadr2.ravedr5_best_neighbour.source_id = gaiadr2.gaia_source.source_id
where
    gaiadr2.gaia_source.ra between {ra_min} and {ra_max}
    and gaiadr2.gaia_source.dec between {dec_min} and {dec_max}
    and gaiadr2.gaia_source.parallax between {parallax_min} and {parallax_max}
""".format(**cutparams)

%store -r g10_region_ravedr5
try:
    g10_region_ravedr5
    print("Loaded g10_region_ravedr5 from cache {:d} rows".format(len(g10_region_ravedr5)))
except NameError:
    print("Querying Gaia archive")
    print(query_region_ravedr5)
    j = gaia.query(query_region_ravedr5)
    g10_region_ravedr5 = table_to_pandas(j)
    %store g10_region_ravedr5

Loaded g10_region_ravedr5 from cache 3413 rows


In [34]:
# join photometry
allwise_columns_keep = [
    'designation','source_id','allwise_oid',
    'w1mpro', 'w1mpro_error', 'w2mpro', 'w2mpro_error', 'w3mpro', 'w3mpro_error', 'w4mpro', 'w4mpro_error',
    'cc_flags','ext_flag','var_flag', 'ph_qual',
]
tmass_columns_keep = [
    'tmass_oid', 'j_m', 'j_msigcom', 'h_m', 'h_msigcom', 'ks_m', 'ks_msigcom',
]

g10_region_allphot = g10_region_phot[allwise_columns_keep+tmass_columns_keep].merge(
    g10_region_ps, on='designation', how='left')\
    .merge(g10_region[['designation','parallax','parallax_error']].reset_index(),
           on='designation', how='right').set_index('index')


g10_region_allphot = g10_region_allphot.rename(columns=dict(
    w1mpro='wise_w1', w1mpro_error='wise_w1_error',
    w2mpro='wise_w2', w2mpro_error='wise_w2_error',
    w3mpro='wise_w3', w3mpro_error='wise_w3_error',
    w4mpro='wise_w4', w4mpro_error='wise_w4_error',
    j_m='tmass_j', j_msigcom='tmass_j_error',
    h_m='tmass_h', h_msigcom='tmass_h_error',
    ks_m='tmass_ks', ks_msigcom='tmass_ks_error',
    g_mean_psf_mag='ps_g', g_mean_psf_mag_error='ps_g_error',
    r_mean_psf_mag='ps_r', r_mean_psf_mag_error='ps_r_error',
    i_mean_psf_mag='ps_i', i_mean_psf_mag_error='ps_i_error',
    z_mean_psf_mag='ps_z', z_mean_psf_mag_error='ps_z_error',
    y_mean_psf_mag='ps_y', y_mean_psf_mag_error='ps_y_error',
))

## Get other names

In [16]:
cutparams = dict(
    parallax_min=8.3, parallax_max=12.4,
    ra_min=g.ra.mean()-20, ra_max=g.ra.mean()+20,
    dec_min=g.dec.mean()-9, dec_max=g.dec.mean()+9
)
query_region_tycho2 = """
select gaiadr2.gaia_source.designation, gaiadr2.gaia_source.source_id,
    tyc.hip, tyc.tyc1, tyc.tyc2, tyc.tyc3, tyc.id_tycho, tyc.id
from gaiadr2.gaia_source
-- tycho2
left join gaiadr2.tycho2_best_neighbour on gaiadr2.tycho2_best_neighbour.source_id = gaiadr2.gaia_source.source_id
left join public.tycho2 tyc on gaiadr2.tycho2_best_neighbour.original_ext_source_id = tyc.id
where
    gaiadr2.gaia_source.ra between {ra_min} and {ra_max}
    and gaiadr2.gaia_source.dec between {dec_min} and {dec_max}
    and gaiadr2.gaia_source.parallax between {parallax_min} and {parallax_max}
""".format(**cutparams)


%store -r g10_region_tycho2
try:
    g10_region_tycho2
    print("Loaded g10_region_tycho2 from cache {:d} rows".format(len(g10_region_tycho2)))
except NameError:
    print("Querying Gaia archive")
    print(query_region_tycho2)
    j = gaia.query(query_region_tycho2)
    g10_region_tycho2 = table_to_pandas(j)
    %store g10_region_tycho2

Loaded g10_region_tycho2 from cache 3414 rows


In [17]:
g10_region_tycho2.count()

designation    3414
source_id      3414
hip             136
tyc1            478
tyc2            478
tyc3            478
id_tycho        478
id              478
dtype: int64

## Check HIP2 only

In [19]:
# cutparams = dict(
#     parallax_min=8.3, parallax_max=12.4,
#     ra_min=g.ra.mean()-20, ra_max=g.ra.mean()+20,
#     dec_min=g.dec.mean()-9, dec_max=g.dec.mean()+9
# )
# query_region_hipparcos2 = """
# select gaiadr2.gaia_source.source_id, hip2.*
# from gaiadr2.gaia_source
# -- hipparcos2
# left join gaiadr2.hipparcos2_best_neighbour on gaiadr2.hipparcos2_best_neighbour.source_id = gaiadr2.gaia_source.source_id
# left join public.hipparcos_newreduction hip2 on gaiadr2.hipparcos2_best_neighbour.original_ext_source_id = hip2.hip
# where
#     gaiadr2.gaia_source.ra between {ra_min} and {ra_max}
#     and gaiadr2.gaia_source.dec between {dec_min} and {dec_max}
#     and gaiadr2.gaia_source.parallax between {parallax_min} and {parallax_max}
# """.format(**cutparams)
# print(query_region_hipparcos2)


# print("Querying Gaia archive")
# j = gaia.query(query_region_hipparcos2)
# g10_region_hipparcos2 = j.to_pandas()
# %store g10_region_hipparcos2

In [20]:
# g10_region_hipparcos2.count()[['source_id','hip']]

In [24]:
figsymbols = dict(
    ra='R.A.',
    dec='Decl.',
    pmra=r'$\mu_\alpha$',
    pmdec=r'$\mu_\delta$',
    parallax=r'$\pi$',
    deg=r'deg',
    masyr=r'$\mathrm{mas}\,\mathrm{yr}^{-1}$')

# External Catalogs

In [25]:
def load_DH2015_table5():
    fn = '../data/dh2015_table5.txt'
    if not os.path.exists(fn):
        from astroquery.vizier import Vizier
        Vizier.ROW_LIMIT = False
        Vizier.columns = ['**']    # get all columns
        # David & Hillenbrand 2015
        dht5 = Vizier.get_catalogs('J/ApJ/804/146/table5')[0]
        dht5.write(fn, format='ascii')
    else:
        dht5 = Table.read(fn, format='ascii')
    return table_to_pandas(dht5)

print('Load David & Hillenbrand 2015 table 5 to `dht5`')
dht5 = load_DH2015_table5()

Load David & Hillenbrand 2015 table 5


In [ ]:
# from ezpadova import parsec

# parsec_solar_gaia = parsec.get_t_isochrones(6, 9, 0.05, 0.0152, phot='gaia', model='parsec12s').to_pandas()
# print(len(parsec_solar_gaia))

# parsec_solar_gaia = parsec_solar_gaia.rename(columns={'log(age/yr)':'log_age'})
# parsec_solar_gaia['bp_rp'] = parsec_solar_gaia['G_BP'] - parsec_solar_gaia['G_RP']



# parsec_solar_tmass = parsec.get_t_isochrones(
#     6, 9, 0.05, 0.0152,
#     phot='2mass_spitzer_wise', model='parsec12s').to_pandas()
# parsec_solar_tmass = parsec_solar_tmass.rename(columns={'log(age/yr)':'log_age'})

# parsec_solar_ps = parsec.get_t_isochrones(6, 9, 0.05, 0.0152, phot='panstarrs1', model='parsec12s').to_pandas()
# parsec_solar_ps = parsec_solar_ps.rename(columns={'log(age/yr)':'log_age'})

# common_columns = list(parsec_solar_gaia.columns.intersection(parsec_solar_tmass.columns))
# parsec_solar = parsec_solar_gaia\
#     .merge(parsec_solar_tmass, on=common_columns)\
#     .merge(parsec_solar_ps, on=common_columns)

In [42]:
# %store parsec_solar

Stored 'parsec_solar' (DataFrame)


In [13]:
%%time
class RXS:
    fn = '../data/cat2rxs.fits'
    t_2rxs = Table.read('../data/cat2rxs.fits')
    print('{:d} in 2RXS'.format(len(t_2rxs)))
    
    coords_2rxs = coords.SkyCoord(t_2rxs['RA_DEG'].data*u.deg, t_2rxs['DEC_DEG'].data*u.deg)

135118 in 2RXS
CPU times: user 8.46 s, sys: 600 ms, total: 9.06 s
Wall time: 9.33 s


fx = ((5.30*(viz.hr1) + 8.31)*1e-12)*viz.count
efx = sqrt( ( ros[gneed].fx / viz.count )^2*viz.e_count^2 + ( 5.3d-12 * viz.count )^2*viz.e_hr1^2 )
lx = 1.2d38*sros.fx*(1d3/gs.PLX)^elx = 1.2d38*sqrt( ((1d3/gs.PLX)^2)^2*sros.efx^2 + (sros.fx*(1d3/gs.PLX)*2.)^2*(1d3/gs.PLX^2*gs.EPLX)^2)
log_lx = alog10(lx)
elog_lx = elx / lx
